In [3]:
!pip install datasets

In [4]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=f2c7245c55c974aad74c37f8fe97cf4c3f00fd692943531a4d59b0f2912d4f6b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [5]:
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import evaluate

In [7]:
!pip install evaluate

In [19]:
!pip install datasets


In [3]:
!pip install rouge_score

In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.8 MB/s eta 0:00:00


In [11]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset
import evaluate

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small').to(device)

# Load and preprocess dataset
dataset = load_dataset("cnn_dailymail", '3.0.0', split='train[:1%]')

def preprocess_function(examples):
    inputs = examples['article']
    targets = examples['highlights']

    model_inputs = tokenizer(inputs, max_length=1024, padding="max_length", truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, padding="max_length", truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=1000,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

# Train the model
trainer.train()




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Step,Training Loss
10,11.826100
20,6.629700
30,3.853600
40,2.533000
50,1.934500
60,1.757400
70,1.807600
80,1.405300
90,1.497000
100,1.361800


TrainOutput(global_step=1436, training_loss=1.2330976223215087, metrics={'train_runtime': 327.3539, 'train_samples_per_second': 8.77, 'train_steps_per_second': 4.387, 'total_flos': 777132624052224.0, 'train_loss': 1.2330976223215087, 'epoch': 1.0})

In [44]:
def generate_summary(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True, padding="max_length")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

user_input = input("Enter text for summarization: ")
summary = generate_summary(user_input, model, tokenizer)
print(f"Summary: {summary}")


Enter text for summarization: Automatic text summarization (or document summarization) is a natural language processing (NLP) method that condenses information from one or more input text documents into an original output text. How much of the input text appears in the output is debated—some definitions state only 10%, others 50%.1 Text summarization algorithms often use deep learning architectures—specifically, transformers—to parse documents and generate text summaries.
Summary: Automatic text summarization (or document summarization) is a natural language processing method that condenses information from one or more input text documents into an original output text.
How little of the input text appears in the output is debated.


In [17]:
# Evaluate fine-tuned model
predictions, references = evaluate_model(model, tokenizer, tokenized_dataset)
# Load pre-trained model
pretrained_model = T5ForConditionalGeneration.from_pretrained('t5-small').to(device)
# Evaluate pre-trained model
pretrain_predictions, pretrain_references = evaluate_model(pretrained_model, tokenizer, tokenized_dataset)
rouge_metric = evaluate.load('rouge')
def compute_rouge(predictions, references):
    results = rouge_metric.compute(predictions=predictions, references=references)
    return results
# Compute ROUGE scores
fine_tuned_rouge = compute_rouge(predictions, references)
pretrain_rouge = compute_rouge(pretrain_predictions, pretrain_references)
print("Fine-tuned model ROUGE scores:", fine_tuned_rouge)
print("Pre-trained model ROUGE scores:", pretrain_rouge)


Fine-tuned model ROUGE scores: {'rouge1': 0.32299982029610985, 'rouge2': 0.12333818306562552, 'rougeL': 0.23089203772123318, 'rougeLsum': 0.2776877445307453}
Pre-trained model ROUGE scores: {'rouge1': 0.2741284809866087, 'rouge2': 0.10448233644433302, 'rougeL': 0.19808568677288524, 'rougeLsum': 0.24243999811541755}
